In [ ]:
!pip install transformers

define model

In [ ]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer
from sklearn.model_selection import train_test_split

In [ ]:
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

preparing data

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
!gdown 1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW

Downloading...
From (original): https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW
From (redirected): https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW&confirm=t&uuid=336ccc9d-0de5-4fdb-95c2-b97d11d75dd2
To: /content/train_data.csv
100% 635M/635M [00:04<00:00, 143MB/s]


In [ ]:
traindf = pd.read_csv("train_data.csv")

<ipython-input-6-d1f667e36c5a>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  traindf = pd.read_csv("train_data.csv")


In [ ]:
train = traindf[['reviewText', 'overall']]

In [ ]:
undersampler = RandomUnderSampler(sampling_strategy={1: 10000, 2: 10000, 3: 10000, 4: 10000, 5: 10000}, random_state=78)

X, y = undersampler.fit_resample(train.drop(columns=['overall']) , train['overall'])

In [ ]:
input_ids = []
attention_masks = []

for text in X['reviewText']:
   inputs = tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=50,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
   )
   input_ids.append(inputs['input_ids'].flatten())
   attention_masks.append(inputs['attention_mask'].flatten())

input_ids = torch.stack(input_ids)
attention_masks = torch.stack(attention_masks)

In [ ]:
train_input_ids, val_input_ids, train_attention_masks, val_attention_masks = train_test_split(input_ids, attention_masks, test_size=0.2, random_state=42)

In [ ]:
class GPT2Dataset(torch.utils.data.Dataset):
   def __init__(self, input_ids, attention_masks):
      self.input_ids = input_ids
      self.attention_masks = attention_masks

   def __len__(self):
      return len(self.input_ids)

   def __getitem__(self, idx):
      return {
        'input_ids': self.input_ids[idx],
        'attention_mask': self.attention_masks[idx]
      }

In [ ]:
train_dataset = GPT2Dataset(train_input_ids, train_attention_masks)
val_dataset = GPT2Dataset(val_input_ids, val_attention_masks)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)

trainning model

In [ ]:
class GPT2Trainer:

    def __init__(self, model, device):
        self.model = model
        self.device = device

    def train(self, train_loader, val_loader, checkpoint_dir):

      self.model.to(self.device)
      criterion = torch.nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-5)
      best_val_loss = float('inf')

      for epoch in range(4):
          self.model.train()
          total_loss = 0
          total_val_loss = 0
          correct = 0
          correct_val = 0
          total_samples = 0
          total_val_samples = 0

          for batch in train_loader:

            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            labels = input_ids.clone()
            optimizer.zero_grad()

            outputs = self.model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))

            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(logits, dim=-1)
            correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            train_accuracy = correct / total_samples
            train_loss = total_loss / len(train_loader)

            print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

          with torch.no_grad():

            for batch in val_loader:

              input_ids = batch['input_ids'].to(self.device)
              attention_mask = batch['attention_mask'].to(self.device)
              labels = input_ids.clone()

              outputs = self.model(input_ids, attention_mask=attention_mask)
              logits = outputs.logits
              loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))

              total_val_loss += loss.item()
              _, predicted = torch.max(logits, dim=-1)
              correct_val += (predicted == labels).sum().item()
              total_val_samples += labels.size(0)
              val_accuracy = correct_val / total_val_samples
              val_loss = total_val_loss / len(val_loader)

            print(f'Epoch {epoch+1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

          #checkpoint
          if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            torch.save({
              'epoch': epoch,
              'odel_state_dict': self.model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': best_val_loss
            }, os.path.join(checkpoint_dir, f'checkpoint_{epoch+1}.pth'))

            print(f'Saved checkpoint at epoch {epoch+1} with val loss {best_val_loss:.4f}')

In [ ]:
trainer = GPT2Trainer(model, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
checkpoint_dir = '/content/drive/MyDrive'

In [ ]:
import os

In [ ]:
trainer.train(train_loader, val_loader, checkpoint_dir)

Streaming output truncated to the last 5000 lines.
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.0000
Epoch 4, Train Loss: 0.0000, Train Accuracy: 50.

saving the model

In [ ]:
model.save_pretrained('fine_tuned_gpt2-n')
tokenizer.save_pretrained('fine_tuned_gpt2-n')

('fine_tuned_gpt2-n/tokenizer_config.json',
 'fine_tuned_gpt2-n/special_tokens_map.json',
 'fine_tuned_gpt2-n/vocab.json',
 'fine_tuned_gpt2-n/merges.txt',
 'fine_tuned_gpt2-n/added_tokens.json',
 'fine_tuned_gpt2-n/tokenizer.json')

predict test data

In [ ]:
# import torch
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
# import pandas as pd


In [ ]:
# model = GPT2LMHeadModel.from_pretrained('content/fine_tuned_gpt2-n')
# tokenizer = GPT2Tokenizer.from_pretrained('content/fine_tuned_gpt2-n')

In [ ]:
# !gdown 1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB

In [ ]:
# testdf = pd.read_csv("test_data.csv")

In [ ]:
# def predict_scores(reviews):
#     model.eval()
#     predictions = []

#     for text in reviews:

#         inputs = tokenizer(
#             text,
#             return_tensors='pt',
#             padding=True,
#             truncation=True,
#             max_length=512
#         ).to(device)

#         with torch.no_grad():
#             outputs = model(**inputs)

#         logits = outputs.logits
#         probabilities = torch.nn.functional.softmax(logits, dim=1)

#         predicted_score = torch.argmax(probabilities, dim=1).item()
#         predictions.append(predicted_score)

#     return predictions

In [ ]:
# predicted_scores = predict_scores(testdf['reviewText'])

In [ ]:
# df = pd.DataFrame({'predicted': predicted_labels}, axis=1)
# df

In [ ]:
# df.to_csv('q2_submission.csv', index=False)